In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
--2022-01-05 16:04:54--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 52.85.146.93, 52.85.146.59, 52.85.146.3, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|52.85.146.93|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [8]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [9]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [10]:
def doc_len(text):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return len(tokens)


In [17]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_title_text = parquetFile.select("id", "title", "text").rdd
parquetFile.count()

21084

In [18]:
id_title_len_dict = {}

id_name_len = doc_title_text.map(lambda x: (x[0],(x[1],doc_len(x[2]))))
id_title_len_dict = id_name_len.collectAsMap()
print(id_title_len_dict)

{12: ('Anarchism', 4064), 25: ('Autism', 4125), 39: ('Albedo', 1843), 290: ('A', 1000), 303: ('Alabama', 8088), 305: ('Achilles', 4298), 307: ('Abraham Lincoln', 7928), 308: ('Aristotle', 6030), 309: ('An American in Paris', 1015), 316: ('Academy Award for Best Production Design', 4619), 324: ('Academy Awards', 4339), 330: ('Actrius', 248), 332: ('Animalia (book)', 233), 334: ('International Atomic Time', 676), 336: ('Altruism', 3728), 339: ('Ayn Rand', 4177), 340: ('Alain Connes', 299), 344: ('Allan Dwan', 805), 358: ('Algeria', 6584), 359: ('List of Atlas Shrugged characters', 2551), 569: ('Anthropology', 4154), 572: ('Agricultural science', 632), 573: ('Alchemy', 4895), 579: ('Alien', 440), 580: ('Astronomer', 457), 586: ('ASCII', 3370), 590: ('Austin (disambiguation)', 195), 593: ('Animation', 3603), 594: ('Apollo', 9953), 595: ('Andre Agassi', 5304), 597: ('Austroasiatic languages', 2288), 599: ('Afroasiatic languages', 3170), 600: ('Andorra', 5100), 612: ('Arithmetic mean', 726),

In [20]:
import json
with open('id_title_len_dict', 'w') as outfile:
  json.dump(title_id_dict, outfile)

In [21]:
with open('id_title_len_dict') as json_file:
    dict_from_json = json.load(json_file)

In [22]:
print(dict_from_json)

{'12': ['Anarchism', 4064], '25': ['Autism', 4125], '39': ['Albedo', 1843], '290': ['A', 1000], '303': ['Alabama', 8088], '305': ['Achilles', 4298], '307': ['Abraham Lincoln', 7928], '308': ['Aristotle', 6030], '309': ['An American in Paris', 1015], '316': ['Academy Award for Best Production Design', 4619], '324': ['Academy Awards', 4339], '330': ['Actrius', 248], '332': ['Animalia (book)', 233], '334': ['International Atomic Time', 676], '336': ['Altruism', 3728], '339': ['Ayn Rand', 4177], '340': ['Alain Connes', 299], '344': ['Allan Dwan', 805], '358': ['Algeria', 6584], '359': ['List of Atlas Shrugged characters', 2551], '569': ['Anthropology', 4154], '572': ['Agricultural science', 632], '573': ['Alchemy', 4895], '579': ['Alien', 440], '580': ['Astronomer', 457], '586': ['ASCII', 3370], '590': ['Austin (disambiguation)', 195], '593': ['Animation', 3603], '594': ['Apollo', 9953], '595': ['Andre Agassi', 5304], '597': ['Austroasiatic languages', 2288], '599': ['Afroasiatic languages